In [1]:
#Import pandas for dataframes as pd
import pandas as pd

#Import datetime module for manipulating dates and times as dt
import datetime as dt

#Import matplotlib for visualisations
from matplotlib import pyplot as plt

#Import seaborn for visualisations
import seaborn as sns

#Import request to allow for connections to APIs
import requests

#Import json package
import json

In [2]:
#Obtaining data from clinicaltrials.gov API (in two parts due to API restrictions)
url_1 = 'https://clinicaltrials.gov/api/query/study_fields?expr=AREA[StudyType]Interventional+AREA[OverallStatus]Completed+AREA[HealthyVolunteers]No+AREA[PrimaryCompletionDate]RANGE[01/01/2013,%20MAX]&min_rnk=1&max_rnk=1000&fmt=JSON&fields=NCTId,BriefTitle,Condition,ArmGroupInterventionName,LeadSponsorName,LeadSponsorClass,EnrollmentCount,StartDate,PrimaryCompletionDate'
url_2 = 'https://clinicaltrials.gov/api/query/study_fields?expr=AREA[StudyType]Interventional+AREA[OverallStatus]Completed+AREA[HealthyVolunteers]No+AREA[PrimaryCompletionDate]RANGE[01/01/2013,%20MAX]&min_rnk=1001&max_rnk=2000&fmt=JSON&fields=NCTId,BriefTitle,Condition,ArmGroupInterventionName,LeadSponsorName,LeadSponsorClass,EnrollmentCount,StartDate,PrimaryCompletionDate'
r_1 = requests.get(url_1)
r_2 = requests.get(url_2)
json_data_1 = r_1.json()
json_data_2 = r_2.json()

In [3]:
#Checking format and keys of imported data above
print(type(json_data_1))
print(json_data_1.keys())

<class 'dict'>
dict_keys(['StudyFieldsResponse'])


In [4]:
#Further investigating structure of imported data 
print(type(json_data_1['StudyFieldsResponse']))
print(json_data_1['StudyFieldsResponse'].keys())

<class 'dict'>
dict_keys(['APIVrs', 'DataVrs', 'Expression', 'NStudiesAvail', 'NStudiesFound', 'MinRank', 'MaxRank', 'NStudiesReturned', 'FieldList', 'StudyFields'])


In [5]:
#Investigating data important under 'StudyFields' in imported data and printing first row.
print(type(json_data_1['StudyFieldsResponse']['StudyFields']))
print(json_data_1['StudyFieldsResponse']['StudyFields'][0])

<class 'list'>
{'Rank': 1, 'NCTId': ['NCT05917457'], 'BriefTitle': ['Investigation of the Effects of Balance Coordination and Reinforcement Training on Gait, Mobility and Posture in Children With Low Vision'], 'Condition': ['Children With Low Vision'], 'ArmGroupInterventionName': ['Other: balance coordination and strengthening training'], 'LeadSponsorName': ['Pamukkale University'], 'LeadSponsorClass': ['OTHER'], 'EnrollmentCount': ['22'], 'StartDate': ['November 20, 2022'], 'PrimaryCompletionDate': ['March 20, 2023']}


In [6]:
#Append the two imported datasets, i.e. the lists located under 'StudyFields' key in json_data_1 and json_data_2.
trials = json_data_1['StudyFieldsResponse']['StudyFields'] + json_data_2['StudyFieldsResponse']['StudyFields']
print(trials[0])
print(trials[-1])

{'Rank': 1, 'NCTId': ['NCT05917457'], 'BriefTitle': ['Investigation of the Effects of Balance Coordination and Reinforcement Training on Gait, Mobility and Posture in Children With Low Vision'], 'Condition': ['Children With Low Vision'], 'ArmGroupInterventionName': ['Other: balance coordination and strengthening training'], 'LeadSponsorName': ['Pamukkale University'], 'LeadSponsorClass': ['OTHER'], 'EnrollmentCount': ['22'], 'StartDate': ['November 20, 2022'], 'PrimaryCompletionDate': ['March 20, 2023']}
{'Rank': 2000, 'NCTId': ['NCT05405959'], 'BriefTitle': ['Kinesio Tape Application With EDF Technique on Active Trigger Points of Upper Trapezius Muscle'], 'Condition': ['Myofascial Pain Syndrome', 'Myofascial Trigger Point Pain'], 'ArmGroupInterventionName': ['Device: Kinesio tape', 'Device: Sham Kinesio Tape'], 'LeadSponsorName': ['Haydarpasa Numune Training and Research Hospital'], 'LeadSponsorClass': ['OTHER'], 'EnrollmentCount': ['180'], 'StartDate': ['January 2, 2019'], 'PrimaryCo

In [7]:
#Creating a dataframe to work with.
trials_df = pd.DataFrame(trials)


# Exploring the DataFrame

In [8]:
#Look at first 5 rows of data to get a better sense of the dataset.
trials_df.head()

,Rank,NCTId,BriefTitle,Condition,ArmGroupInterventionName,LeadSponsorName,LeadSponsorClass,EnrollmentCount,StartDate,PrimaryCompletionDate
0,1,[NCT05917457],[Investigation of the Effects of Balance Coord...,[Children With Low Vision],[Other: balance coordination and strengthening...,[Pamukkale University],[OTHER],[22],"[November 20, 2022]","[March 20, 2023]"
1,2,[NCT05917366],[Effect of Manipulation on Coccydynia],"[Coccyx Disorder, Pelvic Floor; Weak, Stretch]","[Other: Exercises, Other: Coccyx Manipulation,...",[KTO Karatay University],[OTHER],[46],"[July 16, 2022]","[December 20, 2022]"
2,3,[NCT05915780],[Effect of Aerobic Exercises on Static & Dynam...,[Down Syndrome],"[Device: Cycling, Device: Balance Board]",[Riphah International University],[OTHER],[20],"[May 3, 2023]","[May 21, 2023]"
3,4,[NCT05915650],[Application of PNE in Postoperative Pain Mana...,[Pain],"[Behavioral: Pain Neuroscience Education, Beha...",[Zunyi Medical College],[OTHER],[70],"[December 1, 2020]","[December 1, 2021]"
4,5,[NCT05915624],[Tele Nursing Counseling on Anxiety and Patien...,[Ambulatory Surgical Procedures],[Other: telephone counseiling],[Istanbul University - Cerrahpasa (IUC)],[OTHER],[160],"[September 1, 2022]","[May 30, 2023]"


In [9]:
#Finding out number of data entries, columns, data types, and assessing quality of dataset (no. blank cells)
trials_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Rank                      2000 non-null   int64 
 1   NCTId                     2000 non-null   object
 2   BriefTitle                2000 non-null   object
 3   Condition                 2000 non-null   object
 4   ArmGroupInterventionName  2000 non-null   object
 5   LeadSponsorName           2000 non-null   object
 6   LeadSponsorClass          2000 non-null   object
 7   EnrollmentCount           2000 non-null   object
 8   StartDate                 2000 non-null   object
 9   PrimaryCompletionDate     2000 non-null   object
dtypes: int64(1), object(9)
memory usage: 156.4+ KB


In [10]:
#Enrollment should be an integer but must first convert to string and remove unwanted characters.
trials_df['EnrollmentCount'] = trials_df['EnrollmentCount'].astype('string')
trials_df['EnrollmentCount'] = trials_df['EnrollmentCount'].str.replace("[","").str.replace("]","").str.strip("'")
trials_df.head(2)

C:\Users\muire\AppData\Local\Temp\ipykernel_18020\2986588612.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  trials_df['EnrollmentCount'] = trials_df['EnrollmentCount'].str.replace("[","").str.replace("]","").str.strip("'")


,Rank,NCTId,BriefTitle,Condition,ArmGroupInterventionName,LeadSponsorName,LeadSponsorClass,EnrollmentCount,StartDate,PrimaryCompletionDate
0,1,[NCT05917457],[Investigation of the Effects of Balance Coord...,[Children With Low Vision],[Other: balance coordination and strengthening...,[Pamukkale University],[OTHER],22,"[November 20, 2022]","[March 20, 2023]"
1,2,[NCT05917366],[Effect of Manipulation on Coccydynia],"[Coccyx Disorder, Pelvic Floor; Weak, Stretch]","[Other: Exercises, Other: Coccyx Manipulation,...",[KTO Karatay University],[OTHER],46,"[July 16, 2022]","[December 20, 2022]"


In [11]:
#Converting Enrollment to an integer and checking the types.
trials_df['EnrollmentCount'] = trials_df['EnrollmentCount'].astype(int)
trials_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Rank                      2000 non-null   int64 
 1   NCTId                     2000 non-null   object
 2   BriefTitle                2000 non-null   object
 3   Condition                 2000 non-null   object
 4   ArmGroupInterventionName  2000 non-null   object
 5   LeadSponsorName           2000 non-null   object
 6   LeadSponsorClass          2000 non-null   object
 7   EnrollmentCount           2000 non-null   int32 
 8   StartDate                 2000 non-null   object
 9   PrimaryCompletionDate     2000 non-null   object
dtypes: int32(1), int64(1), object(8)
memory usage: 148.6+ KB


In [12]:
#Describing integer data types.
trials_df.describe()

,Rank,EnrollmentCount
count,2000.000000,2000.000000
mean,1000.500000,531.423500
std,577.494589,8379.047502
min,1.000000,1.000000
25%,500.750000,30.000000
50%,1000.500000,58.000000
75%,1500.250000,94.000000
max,2000.000000,262085.000000


# Cleaning the Dataframe

In [13]:
#Confirming that there is no missing data. 
trials_df.isna().any()

Rank                        False
NCTId                       False
BriefTitle                  False
Condition                   False
ArmGroupInterventionName    False
LeadSponsorName             False
LeadSponsorClass            False
EnrollmentCount             False
StartDate                   False
PrimaryCompletionDate       False
dtype: bool

# Sorting the Data

In [16]:
#I want to sort by the length of the study so I need a new column with length of study to work with.

In [17]:
#Having seen the datatypes are not datetime, I want to convert now. Reference solution here.

trials_df["StartDate"] = trials_df["StartDate"].apply(lambda x: pd.to_datetime(x[0]))
trials_df["PrimaryCompletionDate"] = trials_df["PrimaryCompletionDate"].apply(lambda y: pd.to_datetime(y[0]))
print(trials_df.dtypes)

Rank                                 int64
NCTId                               object
BriefTitle                          object
Condition                           object
ArmGroupInterventionName            object
LeadSponsorName                     object
LeadSponsorClass                    object
EnrollmentCount                      int32
StartDate                   datetime64[ns]
PrimaryCompletionDate       datetime64[ns]
dtype: object


In [18]:
#Adding column to capture length of the study as number of days.
trials_df["StudyLength"] = (trials_df["PrimaryCompletionDate"] - trials_df["StartDate"]).dt.days
trials_df.head()

,Rank,NCTId,BriefTitle,Condition,ArmGroupInterventionName,LeadSponsorName,LeadSponsorClass,EnrollmentCount,StartDate,PrimaryCompletionDate,StudyLength
0,1,[NCT05917457],[Investigation of the Effects of Balance Coord...,[Children With Low Vision],[Other: balance coordination and strengthening...,[Pamukkale University],[OTHER],22,2022-11-20,2023-03-20,120
1,2,[NCT05917366],[Effect of Manipulation on Coccydynia],"[Coccyx Disorder, Pelvic Floor; Weak, Stretch]","[Other: Exercises, Other: Coccyx Manipulation,...",[KTO Karatay University],[OTHER],46,2022-07-16,2022-12-20,157
2,3,[NCT05915780],[Effect of Aerobic Exercises on Static & Dynam...,[Down Syndrome],"[Device: Cycling, Device: Balance Board]",[Riphah International University],[OTHER],20,2023-05-03,2023-05-21,18
3,4,[NCT05915650],[Application of PNE in Postoperative Pain Mana...,[Pain],"[Behavioral: Pain Neuroscience Education, Beha...",[Zunyi Medical College],[OTHER],70,2020-12-01,2021-12-01,365
4,5,[NCT05915624],[Tele Nursing Counseling on Anxiety and Patien...,[Ambulatory Surgical Procedures],[Other: telephone counseiling],[Istanbul University - Cerrahpasa (IUC)],[OTHER],160,2022-09-01,2023-05-30,271


In [19]:
#df.sort_values("study_length") and df.sort_values("study_length", ascedning=False) for descending values
#sort by mutliple values here, study length and number enrolled. df.sort_values(["study_length", "enrollment"])
#can subset columns by saying df["columnName"] or multiple columns via df[["columnName1", "columnName2"]] May want to do this to create a new df ad remove irrelevant columns
#can subset rows with a condition like df[df["enrollment"] > 50] or df[df["condition"] == "cancer"] to see if any additional information of interest